In [ ]:
using Oscar

In [ ]:
cd("..")

In [ ]:
currentDir = pwd() # make sure you are running from the main directory.
include(joinpath(currentDir, "src/fileHandling.jl"))
include(joinpath(currentDir, "src/isolate3Lines.jl"))
include(joinpath(currentDir, "src/matroid_realization.jl"))
include(joinpath(currentDir, "src/Jacobian_Criterion.jl"))

First we isolate those that satisfy the three lines property, i.e., every element of the ground set is contained in at least 3 lines (cyclic flats of rank 2). 

In [ ]:
db = Polymake.Polydb.get_db();
collection = db["Matroids.Small"];
d3n11 = Polymake.Polydb.find(collection, Dict("RANK" => 3, "SIMPLE"=>true, "N_ELEMENTS"=>11));


n3C11 = subsets(collect(1:10), 3);
n3C11 = sort(n3C11, by =  x-> reverse(x));

lines_3 = []
for c in d3n11
    Q = Matroid(c)
    ns = count_3_lines_thru_all_points(Q)
    if length(ns) == 0
        continue
    end    
    if minimum(ns) >= 3
        push!(lines_3, to_revlex(Q, n3C11))
    end
end

This data is precomputed in `d3n11/3lines_d3n11.dat`.

In [ ]:
lines_3_precomputed = vec(readlines(joinpath(currentDir, "d3n11/3lines_d3n11.dat")));

In [ ]:
Set(lines_3) == Set(lines_3_precomputed)

The file `d3n11/not_realizable.dat` contains those matroids in `d3n11/3lines_d3n11.dat` that are not realizable over $\mathbb{Q}$. 

In [ ]:
not_realizable = vec(readlines(joinpath(currentDir, "d3n11/nonrealizable.dat")));

**Warning:** the cell below takes ~10 minutes to complete. 

In [ ]:
test_nonrealizable = []

for Qstr in not_realizable
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 11) 
    As = rank_plus1_circuits(Q) 
    MR = new_matroid_realization_space(Q, As[1]; F=QQ, saturate=true)
    push!(test_nonrealizable, !MR.representable)
end

In [ ]:
all(test_nonrealizable)

The files `d3n11/zero_ideal.dat`, `d3n11/univariate_ideal.dat`, `d3n11/principal_ideal.dat`, `d3n11/_not_principal_ideal.dat` record matroids and a maximal circuit. A pair of matroid and circuit is recorded as a `*/0` vector. The first 11 entries record the circuit (e.g., `**0*00*0000` is the circuit `[1,2,4,7]`) and the rest record the matroid in the usual revlex basis encoding. 

**Note:** One could check that these are smooth using the `singular_locus` function in `src/Jacobian_Criterion.jl`. To make this computation more transparent, we carry out the smoothness check explicitly.

The files `d3n11/zero_ideal.dat` contains those matroids in `d3n11/3lines_d3n11.dat` and a maximal circuit such that the ideal of the realization space is zero. Therefore these realization spaces are smooth. 

**Warning:** this cell takes (fill in, something like 15 min) minutes to run.

In [ ]:
zero_ideal = joinpath(currentDir, vec(readlines("d3n11/zero_ideal.dat")))

test_zero = []
for cir_Qstr in zero_ideal
    A = [i for i in 1:11 if string(cir_Qstr[i]) == "*"]     
    Qstr = cir_Qstr[12:176]
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 11) 
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=false)
        
    MR = reduce_ideal_full(MR)
    I = MR.defining_ideal
    push!(test_zero, iszero(I))
end

In [ ]:
all(test_zero)

The files `d3n11/univariate_ideal.dat` contains those matroids in `d3n11/3lines_d3n11.dat` and a maximal circuit such that the ideal of the realization space is univariate. These realization spaces are smooth provided the discriminant is nonzero. 

Missing, find appropiate circuit: `0******0******0**********0****************0************0********0*****************************0********0***************0************************0***0*********0******`

**Warning:** this cell takes (fill in, something like 15 min) minutes to run.

In [ ]:
univariate_ideal = joinpath(currentDir, vec(readlines("d3n11/univariate_ideal.dat")))

test_univariate = []
for cir_Qstr in univariate_ideal
    A = [i for i in 1:11 if string(cir_Qstr[i]) == "*"]     
    Qstr = cir_Qstr[12:176]
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 11) 
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)
        
    MR = reduce_ideal_full(MR)
    I = MR.defining_ideal
    length_vs = length(ideal_vars(gens(I))) 
    push!(test_univariate, isone(length_vs))
end

In [ ]:
all(test_univariate)

The files `d3n11/principal_ideal.dat` contains those matroids in `d3n11/3lines_d3n11.dat` and a maximal circuit such that the ideal of the realization space is principal. By the Jacobian criterion, the affine scheme $\mathsf{Spec}(S^{-1}\mathbb{Q}[x_1,\ldots,x_n] / \langle f \rangle)$ is smooth if and only if the ideal

$$
J = \langle f, \frac{\partial f}{\partial x_1}, \ldots, \frac{\partial f}{\partial x_n}\rangle
$$

is the unit ideal in $S^{-1} \mathbb{Q}[x_1,\ldots,x_n]$. This is equivalent to the saturation $(J:S^{\infty})$ being the unit ideal in $\mathbb{Q}[x_1,\ldots,x_n]$. We perform this explicit check here. 

In [ ]:
principal_ideal = joinpath(currentDir, vec(readlines("d3n11/univariate_ideal.dat")))

test_principal = []
for cir_Qstr in principal
    A = [i for i in 1:11 if string(cir_Qstr[i]) == "*"]     
    Qstr = cir_Qstr[12:176]
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 11) 
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)
        
    MR = reduce_ideal_full(MR)
    R = MR.ambient_ring
    x = gens(R)
    I = MR.defining_ideal
    Igens = gens(I)
    length(Igens) != 1 && error("not principal") 
    JM = jacobian_matrix(R, x, Igens)
    nr, nc = size(JM) 
    J = I + ideal(R, [JM[1,c] for c in 1:nc])
    Sing = stepwise_saturation(J, MR.inequations)
    #Sing = realization_space_2_singular_locus(MR)
    push!(test_principal, isone(Sing))
end

In [ ]:
all(test_principal)